In [1]:
import os
import sys
import copy
import logging
import pandas as pd
from sklearn.model_selection import KFold,GridSearchCV,cross_val_score
#import xgboost as xgb
from catboost import CatBoostRegressor
import catboost
from catboost import CatBoostClassifier, Pool, cv
#from sklearn.metrics import accuracy_score

In [2]:
import sys
print(sys.getrecursionlimit())
sys.setrecursionlimit(10000)

3000


In [3]:
!pip install  catboost

In [4]:
path_to_repo = os.path.dirname(os.getcwd())
path_to_repo

'/Users/daisydu/Desktop'

### $\bullet$ Base model

In [5]:
path_to_dataset = os.path.join(path_to_repo, "machine learning", 'dataset', '')
print(path_to_dataset)

/Users/daisydu/Desktop/machine learning/dataset/


In [6]:
import chardet
import pandas as pd

with open(path_to_dataset+'creditcard_processed.csv', 'rb') as f:
    enc = chardet.detect(f.read())
    
df = pd.read_csv(path_to_dataset+'creditcard_processed.csv', encoding = enc['encoding'])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96696 entries, 0 to 96695
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   month                     96696 non-null  int64  
 1   age                       96696 non-null  int64  
 2   occupation                96696 non-null  int64  
 3   annual_income             96696 non-null  float64
 4   monthly_inhand_salary     96696 non-null  float64
 5   credit_history_age        96696 non-null  int64  
 6   total_emi_per_month       96696 non-null  float64
 7   num_bank_accounts         96696 non-null  int64  
 8   num_credit_card           96696 non-null  int64  
 9   interest_rate             96696 non-null  int64  
 10  num_of_loan               96696 non-null  int64  
 11  delay_from_due_date       96696 non-null  int64  
 12  num_of_delayed_payment    96696 non-null  int64  
 13  changed_credit_limit      96696 non-null  float64
 14  num_cr

In [8]:
def split_vals(df, n): 
    return df[:n].copy(), df[n:].copy()

In [9]:
y = df['credit_score']
df = df.drop(['credit_score'],axis = 1)

In [10]:
n_total = len(df)
n_valid = 12000  
n_train = n_total - n_valid
n_small = 20000

X_train, X_valid = split_vals(df, n_train)
y_train, y_valid = split_vals(y, n_train)

X_small, _ = split_vals(df, n_small)
y_small, _ = split_vals(y, n_small)

print('Number of small training data points: X = {}, y = {}'.format(X_small.shape, y_small.shape))
print('Number of full training data points: X = {}, y = {}'.format(X_train.shape, y_train.shape))
print('Number of validation data points: X = {}, y = {}'.format(X_valid.shape, y_valid.shape))

Number of small training data points: X = (20000, 22), y = (20000,)
Number of full training data points: X = (84696, 22), y = (84696,)
Number of validation data points: X = (12000, 22), y = (12000,)


In [11]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def print_score(m, X_train, y_train, X_valid, y_valid):
    # train set accuracy
    train_pred = m.predict(X_train)
    print('Accuracy on train set: {:.4f}'.format(accuracy_score(y_train, train_pred)))
    # valid set accuracy
    valid_pred = m.predict(X_valid)
    print('Accuracy on valid set: {:.4f}'.format(accuracy_score(y_valid, valid_pred)))
    # train & valid set classification report
    print('Classification Report on train set:\n', classification_report(y_train, train_pred))
    print('Classification Report on valid set:\n', classification_report(y_valid, valid_pred))
    # train & valiid set confusion matrix
    print('Confusion Matrix on train set:\n', confusion_matrix(y_train, train_pred))
    print('Confusion Matrix on valid set:\n', confusion_matrix(y_valid, valid_pred))
    return


In [12]:
help(CatBoostRegressor)

Help on class CatBoostRegressor in module catboost.core:

class CatBoostRegressor(CatBoost)
 |  CatBoostRegressor(iterations=None, learning_rate=None, depth=None, l2_leaf_reg=None, model_size_reg=None, rsm=None, loss_function='RMSE', border_count=None, feature_border_type=None, per_float_feature_quantization=None, input_borders=None, output_borders=None, fold_permutation_block=None, od_pval=None, od_wait=None, od_type=None, nan_mode=None, counter_calc_method=None, leaf_estimation_iterations=None, leaf_estimation_method=None, thread_count=None, random_seed=None, use_best_model=None, best_model_min_trees=None, verbose=None, silent=None, logging_level=None, metric_period=None, ctr_leaf_count_limit=None, store_all_simple_ctr=None, max_ctr_complexity=None, has_time=None, allow_const_label=None, target_border=None, one_hot_max_size=None, random_strength=None, random_score_type=None, name=None, ignored_features=None, train_dir=None, custom_metric=None, eval_metric=None, bagging_temperature=No

In [13]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [14]:
model = CatBoostClassifier(
    n_estimators=500, 
    random_seed=42,
    loss_function='MultiClass',  
    custom_metric=['AUC', 'Accuracy'],  
    logging_level='Silent',  
    class_weights=None,  
    thread_count=-1 
)


In [49]:
model.fit(X_small, y_small, eval_set=(X_small, y_small),plot = True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [16]:
print_score(model, X_small, y_small, X_valid, y_valid)

Accuracy on train set: 0.8804
Accuracy on valid set: 0.6900
Classification Report on train set:
               precision    recall  f1-score   support

           0       0.86      0.89      0.87      5736
           1       0.90      0.87      0.89     10623
           2       0.86      0.89      0.87      3641

    accuracy                           0.88     20000
   macro avg       0.87      0.88      0.88     20000
weighted avg       0.88      0.88      0.88     20000

Classification Report on valid set:
               precision    recall  f1-score   support

           0       0.70      0.61      0.65      3210
           1       0.72      0.76      0.74      6481
           2       0.59      0.61      0.60      2309

    accuracy                           0.69     12000
   macro avg       0.67      0.66      0.66     12000
weighted avg       0.69      0.69      0.69     12000

Confusion Matrix on train set:
 [[5107  595   34]
 [ 847 9273  503]
 [   8  405 3228]]
Confusion Matrix 

In [17]:
predictions = model.predict(X_valid.iloc[:10, :])

In [18]:
predictions

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [19]:
from sklearn.ensemble import RandomForestClassifier

sk_model = RandomForestClassifier(
    n_estimators=500,
    n_jobs=-1, 
    random_state=42,
)

sk_model.fit(X_small, y_small)
print_score(sk_model, X_small, y_small, X_valid, y_valid)


Accuracy on train set: 1.0000
Accuracy on valid set: 0.6948
Classification Report on train set:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      5736
           1       1.00      1.00      1.00     10623
           2       1.00      1.00      1.00      3641

    accuracy                           1.00     20000
   macro avg       1.00      1.00      1.00     20000
weighted avg       1.00      1.00      1.00     20000

Classification Report on valid set:
               precision    recall  f1-score   support

           0       0.71      0.63      0.67      3210
           1       0.74      0.74      0.74      6481
           2       0.58      0.67      0.62      2309

    accuracy                           0.69     12000
   macro avg       0.67      0.68      0.67     12000
weighted avg       0.70      0.69      0.70     12000

Confusion Matrix on train set:
 [[ 5736     0     0]
 [    0 10623     0]
 [    0     0  3641]]
Confusio

### $\bullet$ Learning rate

In [20]:
model = CatBoostClassifier(
    n_estimators=200, 
    random_seed=42,
    learning_rate = 1.0, # learning_rate, eta,
    loss_function='MultiClass',  
    custom_metric=['AUC', 'Accuracy'],  
    logging_level='Silent',  
    class_weights=None,  
    thread_count=-1 
)


In [21]:
model.fit(X_small, y_small, plot=True)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [22]:
print_score(model, X_small, y_small, X_valid, y_valid)

Accuracy on train set: 0.9253
Accuracy on valid set: 0.6411
Classification Report on train set:
               precision    recall  f1-score   support

           0       0.91      0.93      0.92      5736
           1       0.94      0.92      0.93     10623
           2       0.91      0.93      0.92      3641

    accuracy                           0.93     20000
   macro avg       0.92      0.93      0.92     20000
weighted avg       0.93      0.93      0.93     20000

Classification Report on valid set:
               precision    recall  f1-score   support

           0       0.61      0.54      0.57      3210
           1       0.68      0.73      0.70      6481
           2       0.56      0.53      0.55      2309

    accuracy                           0.64     12000
   macro avg       0.62      0.60      0.61     12000
weighted avg       0.64      0.64      0.64     12000

Confusion Matrix on train set:
 [[5347  377   12]
 [ 521 9774  328]
 [  11  245 3385]]
Confusion Matrix 

Let us now try with a smaller learning rate, and more trees to alleviate :

In [23]:
model = CatBoostClassifier(
    n_estimators=5000,  # iterations, n_estimators, num_boost_round, num_trees
    random_seed=42,
    learning_rate = 0.005, # learning_rate, eta,
    loss_function='MultiClass',  
    custom_metric=['AUC', 'Accuracy'],  
    logging_level='Silent',  
    class_weights=None,  
    thread_count=-1 
)


In [24]:
model.fit(X_small, y_small, plot = True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

### $\bullet$ Validation of the number of trees

In [25]:
# see the full list of hyperparameters at 
# https://catboost.ai/docs/concepts/python-reference_catboostregressor.html#python-reference_catboostregressor

model = CatBoostClassifier(
    n_estimators=500, 
    random_seed=42,
    loss_function='MultiClass',  
    custom_metric=['AUC', 'Accuracy'],  
    logging_level='Silent',  
    class_weights=None,  
)


In [26]:
model.fit(X_small, y_small, eval_set = (X_valid, y_valid), plot = True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [27]:
print_score(model, X_small, y_small, X_valid, y_valid)

Accuracy on train set: 0.7792
Accuracy on valid set: 0.6967
Classification Report on train set:
               precision    recall  f1-score   support

           0       0.79      0.73      0.76      5736
           1       0.81      0.80      0.81     10623
           2       0.70      0.78      0.74      3641

    accuracy                           0.78     20000
   macro avg       0.76      0.77      0.77     20000
weighted avg       0.78      0.78      0.78     20000

Classification Report on valid set:
               precision    recall  f1-score   support

           0       0.72      0.62      0.66      3210
           1       0.73      0.75      0.74      6481
           2       0.58      0.65      0.62      2309

    accuracy                           0.70     12000
   macro avg       0.68      0.67      0.67     12000
weighted avg       0.70      0.70      0.70     12000

Confusion Matrix on train set:
 [[4189 1275  272]
 [1115 8541  967]
 [  32  754 2855]]
Confusion Matrix 

In [28]:
model.tree_count_

98

In [29]:
# see the full list of hyperparameters at 
# https://catboost.ai/docs/concepts/python-reference_catboostregressor.html#python-reference_catboostregressor
# and their explanation at
# https://catboost.ai/docs/concepts/python-reference_parameters-list.html#python-reference_parameters-list
model = CatBoostClassifier(
    n_estimators=5000,  # iterations, n_estimators, num_boost_round, num_trees
    random_seed=42,
    learning_rate = 0.005, # learning_rate, eta,
    loss_function='MultiClass',  
    custom_metric=['AUC', 'Accuracy'],  
    logging_level='Silent',  
    class_weights=None,  
    thread_count=-1 
)
model.fit(X_train, y_train, eval_set = (X_valid, y_valid), plot = True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [30]:
print_score(model, X_train, y_train, X_valid, y_valid)

Accuracy on train set: 0.7543
Accuracy on valid set: 0.7248
Classification Report on train set:
               precision    recall  f1-score   support

           0       0.77      0.70      0.73     24275
           1       0.79      0.79      0.79     45022
           2       0.65      0.74      0.69     15399

    accuracy                           0.75     84696
   macro avg       0.74      0.74      0.74     84696
weighted avg       0.76      0.75      0.75     84696

Classification Report on valid set:
               precision    recall  f1-score   support

           0       0.74      0.64      0.68      3210
           1       0.75      0.78      0.77      6481
           2       0.63      0.70      0.67      2309

    accuracy                           0.72     12000
   macro avg       0.71      0.71      0.70     12000
weighted avg       0.73      0.72      0.72     12000

Confusion Matrix on train set:
 [[16972  5866  1437]
 [ 4886 35463  4673]
 [  280  3669 11450]]
Confusio

In [31]:
valid_pool = Pool(X_valid, y_valid)
auc_scores = model.eval_metrics(valid_pool, metrics='AUC', plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

IOStream.flush timed out


We want to draw a _cross-validated_ 

In [32]:
# cv_dataset = Pool(data=X_small, label=y_small)
# 
# params = {
#     "n_estimators": 500, 
#     "loss_function": "MultiClass",
#     "verbose": False,
# }
# 
# scores = catboost.cv(
#     cv_dataset,
#     params,
#     fold_count = 5, 
#     plot = "True",
# )

In [33]:
model = CatBoostClassifier(
    n_estimators = 300, # iterations, n_estimators, num_boost_round, num_trees
    random_seed = 42,
    loss_function = 'MultiClass',
    #allow_writing_files = False,
    logging_level = 'Silent',
)

model.fit(X_small, y_small, plot = True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [34]:
print_score(model, X_small, y_small, X_valid, y_valid)

Accuracy on train set: 0.8825
Accuracy on valid set: 0.6836
Classification Report on train set:
               precision    recall  f1-score   support

           0       0.86      0.89      0.87      5736
           1       0.90      0.88      0.89     10623
           2       0.87      0.89      0.88      3641

    accuracy                           0.88     20000
   macro avg       0.88      0.89      0.88     20000
weighted avg       0.88      0.88      0.88     20000

Classification Report on valid set:
               precision    recall  f1-score   support

           0       0.69      0.60      0.64      3210
           1       0.72      0.75      0.73      6481
           2       0.58      0.60      0.59      2309

    accuracy                           0.68     12000
   macro avg       0.66      0.65      0.66     12000
weighted avg       0.68      0.68      0.68     12000

Confusion Matrix on train set:
 [[5118  594   24]
 [ 847 9299  477]
 [   8  399 3234]]
Confusion Matrix 

### $\bullet$ Hyperparameter tuning

In [35]:
model = CatBoostClassifier(
    n_estimators = 300, # iterations, n_estimators, num_boost_round, num_trees
    random_seed = 42,
    loss_function = 'MultiClass',
    #allow_writing_files = False,
    logging_level = 'Silent',
)

In [36]:
param_grid = [
    {'min_data_in_leaf': [2, 10, 50],  # equivalent of scikit-learn's 'min_samples_leaf'
     'colsample_bylevel': [0.5, 0.75], # equivalent of scikit-learn's max_features
    },
]

In [37]:
grid = GridSearchCV(
    estimator = model, 
    param_grid = param_grid, 
    scoring = None, # uses estimator's default score method
    refit = True, # keep a fitted version of the overall best model
    cv = 5, 
    return_train_score = True,
    verbose = 2
)

grid.fit(X_small, y_small)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] END ..........colsample_bylevel=0.5, min_data_in_leaf=2; total time=   1.3s
[CV] END ..........colsample_bylevel=0.5, min_data_in_leaf=2; total time=   1.6s
[CV] END ..........colsample_bylevel=0.5, min_data_in_leaf=2; total time=   1.8s
[CV] END ..........colsample_bylevel=0.5, min_data_in_leaf=2; total time=   2.1s
[CV] END ..........colsample_bylevel=0.5, min_data_in_leaf=2; total time=   1.8s
[CV] END .........colsample_bylevel=0.5, min_data_in_leaf=10; total time=   1.8s
[CV] END .........colsample_bylevel=0.5, min_data_in_leaf=10; total time=   1.7s
[CV] END .........colsample_bylevel=0.5, min_data_in_leaf=10; total time=   1.7s
[CV] END .........colsample_bylevel=0.5, min_data_in_leaf=10; total time=   1.7s
[CV] END .........colsample_bylevel=0.5, min_data_in_leaf=10; total time=   1.7s
[CV] END .........colsample_bylevel=0.5, min_data_in_leaf=50; total time=   1.7s
[CV] END .........colsample_bylevel=0.5, min_data

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x303aebd10>,
             param_grid=[{'colsample_bylevel': [0.5, 0.75],
                          'min_data_in_leaf': [2, 10, 50]}],
             return_train_score=True, verbose=2)

In [38]:
best_model = grid.best_estimator_
best_model

In [39]:
params = best_model.get_all_params()

print('min_data_in_leaf:', params['min_data_in_leaf'])

min_data_in_leaf: 2


In [40]:
print_score(best_model, X_small, y_small, X_valid, y_valid)

Accuracy on train set: 0.8794
Accuracy on valid set: 0.6923
Classification Report on train set:
               precision    recall  f1-score   support

           0       0.86      0.89      0.87      5736
           1       0.90      0.87      0.89     10623
           2       0.86      0.88      0.87      3641

    accuracy                           0.88     20000
   macro avg       0.87      0.88      0.88     20000
weighted avg       0.88      0.88      0.88     20000

Classification Report on valid set:
               precision    recall  f1-score   support

           0       0.70      0.60      0.65      3210
           1       0.72      0.77      0.74      6481
           2       0.60      0.60      0.60      2309

    accuracy                           0.69     12000
   macro avg       0.67      0.66      0.66     12000
weighted avg       0.69      0.69      0.69     12000

Confusion Matrix on train set:
 [[5101  600   35]
 [ 840 9281  502]
 [   9  425 3207]]
Confusion Matrix 

In [41]:
# randomized splitting strategy
cv = KFold(n_splits = 5, shuffle = True, random_state = 42)

scores = cross_val_score(best_model, X_small, y_small, cv = cv)

print(scores)
print("r2 is {:.3f} with a standard deviation of {:.3f}".format(scores.mean(), scores.std()))

[0.79625 0.8015  0.7835  0.79425 0.79125]
r2 is 0.793 with a standard deviation of 0.006


### $\bullet$ Feature importance

In [42]:
model = CatBoostClassifier(
    n_estimators = 300, # n_estimators, num_boost_round, num_trees
    random_seed = 42,
    loss_function = 'MultiClass',
    #allow_writing_files = False,
    logging_level = 'Silent',
)

model = model.fit(X_small, y_small, plot = False)

In [43]:
train_pool = Pool(X_train, y_train)

feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse = True):
    print('{}: {}'.format(name, score))

outstanding_debt: 11.722299411120627
credit_mix: 8.284554691919789
interest_rate: 7.593023475819719
delay_from_due_date: 6.924514640725791
age: 6.158017051592623
total_emi_per_month: 6.042167809183885
month: 5.674548927232129
changed_credit_limit: 5.540166636836345
credit_history_age: 5.4827318231927995
num_credit_card: 4.77108403982993
occupation: 4.216142960753792
num_of_delayed_payment: 3.9438391536554773
monthly_inhand_salary: 3.7758045484334195
num_bank_accounts: 3.7364115795424806
num_credit_inquiries: 3.5191828829508913
annual_income: 2.9781041015923
num_of_loan: 2.5089749217750157
monthly_balance: 1.928166015078249
payment_behaviour: 1.7525314544470407
amount_invested_monthly: 1.6868632666251215
credit_utilization_ratio: 1.4098402830807186
payment_of_min_amount: 0.35103032461185496


In [44]:
cols = [n for n, i in zip(feature_names, feature_importances) if i > 0.005]

In [45]:
df_keep = df[cols]

n_total = len(df_keep)
n_valid = 12000
n_train = n_total - n_valid
n_small = 20000

X_train_keep, X_valid_keep = split_vals(df_keep, n_train)
y_train, y_valid = split_vals(y, n_train)

X_small_keep, _ = split_vals(df_keep, n_small)
y_small, _ = split_vals(y, n_small)

print('Number of small training data points: X = {}, y = {}'.format(X_small.shape, y_small.shape))
print('Number of full training data points: X = {}, y = {}'.format(X_train.shape, y_train.shape))
print('Number of validation data points: X = {}, y = {}'.format(X_valid.shape, y_valid.shape))

Number of small training data points: X = (20000, 22), y = (20000,)
Number of full training data points: X = (84696, 22), y = (84696,)
Number of validation data points: X = (12000, 22), y = (12000,)


In [46]:
model = CatBoostClassifier(
    n_estimators = 300, # n_estimators, num_boost_round, num_trees
    random_seed = 42,
    loss_function = 'MultiClass',
    #allow_writing_files = False,
    logging_level = 'Silent',
)

model = model.fit(X_small_keep, y_small, plot = True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [47]:
print_score(model, X_small_keep, y_small, X_valid_keep, y_valid)

Accuracy on train set: 0.8825
Accuracy on valid set: 0.6836
Classification Report on train set:
               precision    recall  f1-score   support

           0       0.86      0.89      0.87      5736
           1       0.90      0.88      0.89     10623
           2       0.87      0.89      0.88      3641

    accuracy                           0.88     20000
   macro avg       0.88      0.89      0.88     20000
weighted avg       0.88      0.88      0.88     20000

Classification Report on valid set:
               precision    recall  f1-score   support

           0       0.69      0.60      0.64      3210
           1       0.72      0.75      0.73      6481
           2       0.58      0.60      0.59      2309

    accuracy                           0.68     12000
   macro avg       0.66      0.65      0.66     12000
weighted avg       0.68      0.68      0.68     12000

Confusion Matrix on train set:
 [[5118  594   24]
 [ 847 9299  477]
 [   8  399 3234]]
Confusion Matrix 

In [51]:
import dill
import os

path_to_model_dir = os.path.join('machine learning',)
path_to_model = os.path.join(path_to_model_dir, 'catboost_model.pk')

os.makedirs(path_to_model_dir, exist_ok=True)

with open(path_to_model, 'wb') as f:
    dill.dump(model, f)
print("Model saved successfully!")


Model saved successfully!
